In [26]:
import pandas as pd
import json
import re
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer

from itertools import product


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/stoffregen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/stoffregen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Load the dataset

In [27]:
data = json.loads(open("../data/processed/Oppositional_thinking_analysis_dataset.json").read())

## Define function to handle imbalance in the training set

In [28]:
def balance_data(data):
    df = pd.DataFrame(data)
    df.pop('id')
    train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['category'], random_state=42)

    # Handle class imbalance in the training set
    train_df_majority = train_df[train_df.category == 'CRITICAL']
    train_df_minority = train_df[train_df.category == 'CONSPIRACY']

    train_df_minority_upsampled = resample(train_df_minority, 
                                        replace=True,     
                                        n_samples=len(train_df_majority),    
                                        random_state=42)

    train_df_balanced = pd.concat([train_df_majority, train_df_minority_upsampled])

    return train_df_balanced, test_df



In [29]:
train_df_balanced, test_df = balance_data(data)

# Define different complex pre-processing functions

In [30]:
def preprocess_basic(text, lem_tag = True, stem_tag = False):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^A-Za-z\s]', '', text)
    tokens = nltk.word_tokenize(text.lower())

    if lem_tag:
    # Lemmatization
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]

    if stem_tag:
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(word) for word in tokens]

    return ' '.join(tokens)



def preprocess_advanced(text:str, lem_tag = True, stem_tag = False) -> int:

    # Lowercasing
    text = text.lower()
    text = re.sub(r'\d+', '', text) # remove decimals  
    text = re.sub(r'[\:\-\']', '', text)  # Remove specific punctuation
    text = re.sub(r'http\S+', '', text) # Remove URLs
    text = re.sub(r'\s+', ' ', text) # Remove extra whitespace
    text = re.sub(r'[^\w\s]', '', text) # Remove special characters
    text = re.sub(r'\d+\.\d+', '', text)  # Matches one or more digits followed by a dot and one or more digits
    text = re.sub(r'\bcom\b', '', text, flags=re.IGNORECASE)  # Matches "com" at word boundaries (whole word)


    # Tokenization
    tokens = word_tokenize(text)

    # Removing stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    if lem_tag:
    # Lemmatization
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    if stem_tag:
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(word) for word in tokens]

    return ' '.join(tokens)

## Define train and evaluate function

In [96]:
def train_and_evaluate(train_df, test_df, pipeline):
    # print trian_df columns
    print(f"Columns in train_df: {train_df.columns}")
    # put all available features into X_train, drop columns category and id if they exist
    drop_columns = ['category'] if 'category' in train_df.columns else []
    drop_columns += ['id'] if 'id' in train_df.columns else []
    X_train = train_df.drop(drop_columns, axis=1)
    # If only one column remains, convert it to a Series
    if len(X_train.columns) == 1:
        X_train = X_train.iloc[:, 0]
        
    y_train = train_df['category']
    
    X_test = test_df['text']
    y_test = test_df['category']
    
    # print shapes
    print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    report = classification_report(y_test, y_pred, output_dict=True)    
    # print f1 score
    print(f"f1 score {pipeline.named_steps['vectorizer'].__class__.__name__}: {report['weighted avg']['f1-score']}")
    
    # return the report as a dataframe
    report_df = pd.DataFrame(report).transpose()
    return report_df

## Define function to save the results into a csv file

In [97]:
def save_results_to_csv(report, base_path = '../reports/', file_name = 'classification_report.csv'):
    # append the data
    report.to_csv(base_path + file_name, mode='a', header=False)

## Define the basic preprocessor class 

In [98]:
class TextPreprocessor(BaseEstimator, TransformerMixin):
    
    def __init__(self, custom_preprocess_function, use_lemmatization=True, use_stemmanization = True):
        self.use_lemmatization = use_lemmatization
        self.use_stemmanization = use_stemmanization
        self.preprocess_function = custom_preprocess_function

    # define fit function to make the transformer pipeline compatible with sklearn
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        res = [self.preprocess_function(text, self.use_lemmatization, self.use_stemmanization) for text in X]
        return res


## Define function to create pipelines with:
* custom pre-processing functions
* custom classifier (e.g. naive bayes, FNN)
* no lemmatization nor stemming
* ngram range  


In [99]:
def create_pipelines(ngram_lower_bound, ngram_upper_bound, custom_preprocess, classifier):
    vectorizers = [
        (CountVectorizer, 'CountVectorizer'),
        (TfidfVectorizer, 'TfidfVectorizer')
    ]
    
    pipelines = {}
    
    for vectorizer, vec_name in vectorizers:
        pipeline_name = (
            f'{vec_name} with preprocessing function {custom_preprocess.__name__}, '
            f'classifier {type(classifier).__name__} no Lemmatization and ngram: '
            f'{ngram_lower_bound}, {ngram_upper_bound}'
        )
        
        pipelines[pipeline_name] = Pipeline([
            ('preprocessor', TextPreprocessor(custom_preprocess_function=custom_preprocess, use_lemmatization=False, use_stemmanization=False)),
            ('vectorizer', vectorizer(ngram_range=(ngram_lower_bound, ngram_upper_bound))),
            ('classifier', classifier)
        ])
    
    return pipelines


## Train and evaluate naive bayes and FNN models with different preprocessing
* without lemmatization nor stemming
* for ngram range 1-3

In [ ]:
report_file_name = 'classification_report_1.csv'
# empty report
report_df = pd.DataFrame()
report_df.to_csv('../reports/' + report_file_name)


for preprocess_function in [preprocess_advanced, preprocess_basic]:
    for classifier in [MultinomialNB(), MLPClassifier(hidden_layer_sizes=(50,))]:
        for ngram_lower_bound, ngram_upper_bound in product(range(1, 3), repeat=2):
            if ngram_lower_bound <= ngram_upper_bound:
                pipelines = create_pipelines(ngram_lower_bound, ngram_upper_bound, preprocess_function, classifier)
                for name, pipeline in pipelines.items():
                    print(f"Evaluating {name} with ngram range: {ngram_lower_bound}, {ngram_upper_bound}...")
                    report_df = train_and_evaluate(train_df_balanced, test_df, pipeline)
                    # enrich df with the name of the pipeline, ngram range and preprocess function
                    report_df['pipeline'] = name
                    report_df['classifier'] = pipeline.named_steps['classifier'].__class__.__name__
                    report_df['vectorizer'] = pipeline.named_steps['vectorizer'].__class__.__name__
                    report_df['ngram_lower_bound'] = ngram_lower_bound
                    report_df['ngram_upper_bound'] = ngram_upper_bound
                    report_df['preprocess_function'] = preprocess_function.__name__
                    save_results_to_csv(report_df, file_name=report_file_name)
                    

## Enrich data json with uppercase percentage and comment length feature

In [86]:
## add feature uppercase percentage
def calculate_uppercase_percentage(text):
    uppercase_count = 0
    total_letters = 0
    
    for char in text:
        if char.isalpha():  # Check if the character is a letter
            total_letters += 1
            if char.isupper():  # Check if the letter is uppercase
                uppercase_count += 1
    
    if total_letters == 0:
        return 0
    uppercase_percentage = (uppercase_count / total_letters) * 100
    
    return uppercase_percentage

def classify_uppercase_percentage(percentage):
    if percentage < 6:
        return "low"
    elif 6 <= percentage <= 12:
        return "neutral"
    else:
        return "high"
    
# add custom feature: comment length
def classify_comment_lenght(length):
    if length < 190:
        return "short"
    elif 190 <= length <= 560:
        return "average"
    else:
        return "long"

# Update the data with the new key-value pair
updated_data = data.copy()
for comment in data:
    comment_length = classify_comment_lenght(len(comment["text"]))
    uppercase_percentage = calculate_uppercase_percentage(comment["text"])
    uppercase_amount = classify_uppercase_percentage(uppercase_percentage)
    comment["comment_length"] = comment_length
    comment["uppercase_amount"] = uppercase_amount


# Save the updated data to a new file
with open('../data/processed/Oppositional_thinking_analysis_dataset_with_features.json', 'w') as f:
    json.dump(updated_data, f, indent=4)


In [104]:
data = json.loads(open("../data/processed/Oppositional_thinking_analysis_dataset_with_features.json").read())
train_df_balanced, test_df = balance_data(data)


## Add a preprocessor to handle the new features
* it uses a OneHotEncoder to encode the custom features

In [105]:
class CombinedFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, text_transformer, ngram_range = (1,1),  additional_features = []):
        self.text_transformer = text_transformer
        self.additional_features = additional_features
        self.encoder = OneHotEncoder()
        self.vectorizer = CountVectorizer(ngram_range=ngram_range)


    def fit(self, X, y=None):
        # Fit the individual transformers
        #self.text_transformer.fit(X['text'])
        if self.additional_features:
            self.encoder.fit(X[self.additional_features]) 

        return self

    def transform(self, X):
        text_features = self.text_transformer.transform(X['text'])
     
        if self.additional_features:
            additional_features = self.encoder.transform(X[self.additional_features]).toarray()
            text_features = np.hstack((text_features.toarray(), additional_features))
       
        return text_features
    


## We add some new pipelines 
* those use the combined features with either CountVectorizer or TfidfVectorizer
* they use both lemmatization and stemming
* they use a OneHotEncoder for the custom features

In [106]:
# Pre-processing pipelines
custom_preprocess = preprocess_advanced

def create_advanced_pipelines(ngram_lower_bound, ngram_upper_bound, custom_preprocess, classifier, custom_feautures : list ):
    pipelines = {
        f'CountVectorizer with preprocessing function {custom_preprocess.__name__} and no Lemmatization nor stem and ngram: {ngram_lower_bound}, {ngram_upper_bound} with custom features: {custom_feautures}': Pipeline([
            ('preprocessor',  TextPreprocessor(custom_preprocess_function=custom_preprocess, use_stemmanization=False, use_lemmatization=False)),
            ('features', CombinedFeatures(CountVectorizer(ngram_range=(ngram_lower_bound, ngram_upper_bound)), custom_feautures)),
            ('classifier', classifier)
        ]),
        f'TfidfVectorizer with preprocessing function {custom_preprocess.__name__} and no Lemmatization nor stem and ngram: {ngram_lower_bound}, {ngram_upper_bound} with custom features: {custom_feautures}': Pipeline([
            ('preprocessor', TextPreprocessor(custom_preprocess_function=custom_preprocess, use_stemmanization=False, use_lemmatization=False)),
            ('features', CombinedFeatures(TfidfVectorizer(ngram_range=(ngram_lower_bound, ngram_upper_bound)),custom_feautures)),
            ('classifier', classifier)
        ]),
        f'CountVectorizer with preprocessing function {custom_preprocess.__name__} and Lemmatization and stem and ngram: {ngram_lower_bound}, {ngram_upper_bound} with custom features: {custom_feautures}': Pipeline([
            ('preprocessor',  TextPreprocessor(custom_preprocess_function=custom_preprocess, use_stemmanization=True, use_lemmatization=True)),
            ('features', CombinedFeatures(CountVectorizer(ngram_range=(ngram_lower_bound, ngram_upper_bound)), custom_feautures)),
            ('classifier', classifier)
        ]),
        f'TfidfVectorizer with preprocessing function {custom_preprocess.__name__} and Lemmatization and stem and ngram: {ngram_lower_bound}, {ngram_upper_bound} with custom features: {custom_feautures}': Pipeline([
            ('preprocessor', TextPreprocessor(custom_preprocess_function=custom_preprocess, use_stemmanization=True, use_lemmatization=True)),
            ('features', CombinedFeatures(TfidfVectorizer(ngram_range=(ngram_lower_bound, ngram_upper_bound)), custom_feautures)),
            ('classifier', classifier)
        ])
    }   
    return pipelines


## Run and evaluate the advanced pipelines 
* with naive bayes as well as FNN
* use different feature_list combinations

In [107]:
report_file_name = 'classification_report_2.csv'
# empty report
report_df = pd.DataFrame()
report_df.to_csv('../reports/' + report_file_name)

# Train and evaluate models with different pipelines
for classifier in [MultinomialNB(), MLPClassifier(hidden_layer_sizes=(50,))]:
    for feature_list in [['uppercase_amount', 'comment_length'], ['comment_length'], ['uppercase_amount']]:
        for preprocess_function in [preprocess_advanced, preprocess_basic]:
            for ngram_lower_bound in range(1,3):
                for ngram_upper_bound in range(1,3): 
                    if ngram_lower_bound <= ngram_upper_bound:
                        pipelines = create_advanced_pipelines(ngram_lower_bound, ngram_upper_bound, preprocess_function, classifier, feature_list)
                        for name, pipeline in pipelines.items():
                            print(f"Evaluating {name}...")
                            train_and_evaluate(train_df_balanced, test_df, pipeline)
                            # enrich df with the name of the pipeline, ngram range and preprocess function
                            report_df['pipeline'] = name
                            report_df['classifier'] = pipeline.named_steps['classifier'].__class__.__name__
                            report_df['vectorizer'] = pipeline.named_steps['vectorizer'].text_transformer
                            report_df['features'] = pipeline.named_steps['features'].__class__.__name__
                            report_df['ngram_lower_bound'] = ngram_lower_bound
                            report_df['ngram_upper_bound'] = ngram_upper_bound
                            report_df['preprocess_function'] = preprocess_function.__name__
                            report_df['preprocess_function_uses_lemmatization'] = pipeline.named_steps['preprocessor'].use_lemmatization
                            report_df['preprocess_function_uses_stemmanization'] = pipeline.named_steps['preprocessor'].use_stemmanization
                            report_df['feature_list'] = feature_list
                            save_results_to_csv(report_df, file_name=report_file_name)

Evaluating CountVectorizer with preprocessing function preprocess_advanced and no Lemmatization nor stem and ngram: 1, 1 with custom features: ['uppercase_amount', 'comment_length']...
Columns in train_df: Index(['text', 'category', 'comment_length', 'uppercase_amount'], dtype='object')
X_train shape: (4194, 3), y_train shape: (4194,)


TypeError: list indices must be integers or slices, not str